In [1]:
import pandas as pd
from xgboost import XGBClassifier
import pickle
from flask import Flask,request,render_template,jsonify

from static.get_dummies import GetDummies
from werkzeug.utils import secure_filename

In [2]:
# Load the encoder from the file
loaded_enc   = pickle.load(open("static/credit_score_multi_class_ord_encoder.pkl", "rb")) 
loaded_le    = pickle.load(open("static/credit_score_multi_class_le.pkl", "rb"))
loaded_dummy = pickle.load(open("static/credit_score_multi_class_dummy.pkl", "rb"))

# Load the model from the file
loaded_model = XGBClassifier()
loaded_model.load_model("static/credit_score_multi_class_xgboost_model.json")

C:\Users\linhu\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\linhu\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
selected_col = ['Total_EMI_per_month', 'Num_Bank_Accounts', 'Num_of_Delayed_Payment',
 'Delay_from_due_date', 'Changed_Credit_Limit', 'Num_Credit_Card',
 'Outstanding_Debt', 'Interest_Rate', 'Credit_Mix']

response_data = {
    'columns': [],
    'data': []
}

In [4]:
app = Flask(__name__)

In [5]:
@app.route("/",methods = ["GET","POST"])
def home():
    return(render_template("index.html"))

@app.route("/upload",methods = ["GET","POST"])
def upload():
    file = request.files['file']
    filename = secure_filename(file.filename)
    file.save("static/" + filename)

    df_input = pd.read_csv("static/" + filename)
    df_test = df_input[selected_col]
    x_mapping = {"Bad":0, "Good":1, "Standard":2}
    df_test['Credit_Mix'] = df_test['Credit_Mix'].map(x_mapping)

    y_test_pred = loaded_model.predict(df_test)
    y_mapping = {1: "Poor", 2: "Standard", 0:"Good"}
    pred_cat = [y_mapping[value] for value in y_test_pred]

    df_final = df_input.copy()
    df_final['Credit Score'] = pred_cat
    response_data = {
        'columns': df_final.columns.tolist(),
        'data': df_final.values.tolist()
    }
    return jsonify(response_data)
    

In [6]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2024 00:42:03] "GET / HTTP/1.1" 200 -
C:\Users\linhu\AppData\Local\Temp\ipykernel_16504\201074760.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Credit_Mix'] = df_test['Credit_Mix'].map(x_mapping)
127.0.0.1 - - [20/Apr/2024 00:42:06] "POST /upload HTTP/1.1" 200 -
